3rd Place Swahili NLP

1. I use Out-of-Fold Predictions that boost most the score   : https://www.kaggle.com/ipythonx/optimizing-metrics-out-of-fold-weights-ensemble, https://www.kaggle.com/getting-started/177975#988470
2. Different model increase the boost score
3. Something puzzled me in this competition and still didnt figureout is use of GPU for catboost has poor results compared to CPU

# Install Missing Libraries

In [1]:
!pip install googletrans==4.0.0-rc1   # this is the app for language converter
!pip install -q --upgrade catboost   # install catboost algorithms
!pip install transformers --upgrade
!pip install simpletransformers

  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 55 kB 554 kB/s            
     |████████████████████████████████| 42 kB 863 kB/s            
     |████████████████████████████████| 1.3 MB 1.2 MB/s            
     |████████████████████████████████| 133 kB 48.4 MB/s            
     |████████████████████████████████| 58 kB 6.0 MB/s             
     |████████████████████████████████| 65 kB 3.0 MB/s             
     |████████████████████████████████| 53 kB 2.2 MB/s             
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17415 sha256=46694f532d6544a52cde52bd0f17e5ab90bc60c1f90cbaceb8505af55d4588be
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.13.0
    Uninstalling h11-0.13.0:
      Successfully uninstalled h11-0.13.0
  Attempting uninstall: idna
    Fou

Import necessary Libraries

In [2]:
import numpy as np
import pandas as pd
import random
import re, spacy, string
import en_core_web_sm
nlp = en_core_web_sm.load()
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import time

# hide warnings
import warnings
warnings.filterwarnings('ignore')
# set options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



from pprint import pprint
from datetime import datetime
from collections import Counter
from googletrans import Translator
from kaggle_datasets import KaggleDatasets
from dask import bag, diagnostics    # for parallel processing

# Import machine learning algorithms to be used
1. Logistic Classfication
2. Catboost

In [3]:
# important function based on logistic classification

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.utils.class_weight import compute_class_weight
from sklearn import model_selection,metrics, preprocessing

# important function based on catboost

import catboost as cb

from catboost import Pool, CatBoostClassifier

In [4]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/zindiswahili3/sample_submission.csv
/kaggle/input/zindiswahili3/swwiki.csv
/kaggle/input/zindiswahili3/Common Swahili Stop-words.csv
/kaggle/input/zindiswahili3/stopswahili.txt
/kaggle/input/zindiswahili3/swahili2englishTweets.csv
/kaggle/input/zindiswahili3/Train.csv
/kaggle/input/zindiswahili3/Test.csv
/kaggle/input/zindiswahili3/testenglishtweet.csv


# data set link in Kaggle: https://www.kaggle.com/mikemollel/zindiswahili3

In [5]:
# https://www.kaggle.com/mikemollel/zindiswahili3 is the link kaggle dataset
path = KaggleDatasets().get_gcs_path('zindiswahili3')   # path to dataset

Loading dataset

In [6]:
df = pd.read_csv(path+'/Train.csv') # Train data set for ovevsall logistic regression
df_test = pd.read_csv(path+'/Test.csv')  # Test set 
df_submission = pd.read_csv(path+'/sample_submission.csv')   # sample submission file


df_cat = pd.read_csv(path+'/Train.csv') # Train data set for catboost
df_test_cat = pd.read_csv(path+'/Test.csv')  # Test set catboost
df_submission_cat = pd.read_csv(path+'/sample_submission.csv')   # sample submission file catboot

df.head()

,ID,Tweets,Labels
0,2,So chuga si tunakutana kesho kwenye Nyamachoma...,0
1,3,Asante sana watu wa Sirari jimbo la Tarime ...,1
2,5,Leo nimepata kitambulisho changu cha taifa ...,1
3,6,Mgema akisifiwa tembo hulitia maji,0
4,8,Ee Mwenyezi Mungu Msamehe na Umrehemu na Umuaf...,1


In [7]:
TEXT_arg = False # this is to argumented the text however due to speed I argumented and store offline. To do online set True

def back_translate_en(sequence):

    languages = ['en', 'fr', 'th', 'tr', 'ur', 'ru', 'bg', 'de', 'ar', 'zh-cn', 'hi', 'vi', 'es', 'el']
    
    #instantiate translator
    translator = Translator()
    #store original language so we can convert back
    org_lang = translator.detect(sequence).lang   # this works if some tweets are not in Swahili or containts mix lang
    #randomly choose language to translate sequence to  
    random_lang = np.random.choice([lang for lang in languages if lang is not org_lang])
    #translate to new language randomly 
    translated = translator.translate(sequence, dest = random_lang).text # intention is to get to paraphrase
    
    #translate back to english if rand langu is English leave English 
    translated_back = translator.translate(translated, dest = 'en').text
    
    return translated_back

#Test if the conversion works: yes we get paraphrased version and works
for i in range(5):
    output = back_translate_en('Huu moto ni hatari sana kwa maisha ya wanadamu. mimi nitakuja sasa ili twende shule')
    print(output)

This fire is very dangerous for human life. I will come to school now
This fever is very dangerous to human life. I will come to school
This fire is very dangerous to human life.I am going to school now.
This fire is very dangerous for human life. I will come to school.
This fire is very dangerous for human life.I'll come now to go to school


Load paraphrased tweet or convert use the above API
1. Note that this process is time consuming if you want converted online set Text_arg = True else False

In [8]:
if TEXT_arg == False:
    entweet = pd.read_csv(path+'/swahili2englishTweets.csv') # Train data set for ovevsall logistic regression
    df['English_tweet'] = entweet
    df_cat['English_tweet'] = entweet

    
    entweet_test = pd.read_csv(path+'/testenglishtweet.csv')  # Test set catboost
    df_test['English_tweet'] = entweet_test
    df_test_cat['English_tweet'] = entweet_test
    
else:   # this is very trick if activated run only once since google set max time. I recommend to use the arg = False unless if you want for test set true
    translator = Translator()
    prediction_lang = bag.from_sequence(df['Tweets'].tolist()).map(back_translate_en)
    with diagnostics.ProgressBar():
        prediction_list = prediction_lang.compute()
    df['English_tweet'] = prediction_list
    df_cat['English_tweet'] = prediction_list
    
    
    prediction_lang = bag.from_sequence(df_test['Tweets'].tolist()).map(back_translate_en)
    with diagnostics.ProgressBar():
        prediction_list = prediction_lang.compute()
    
    df_test['English_tweet'] = prediction_list
    df_test_cat['English_tweet'] = prediction_list
    

In [9]:
# API for TEXT
import random
import re
from nltk import sent_tokenize
from tqdm import tqdm
from albumentations.core.transforms_interface import DualTransform, BasicTransform


from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

# Modules for Text format etc

In [10]:
class NLPTransform(BasicTransform):
    """ Transform for nlp task."""
    LANGS = {
        'en': 'english'
    }

    @property
    def targets(self):
        return {"data": self.apply}
    
    def update_params(self, params, **kwargs):
        if hasattr(self, "interpolation"):
            params["interpolation"] = self.interpolation
        if hasattr(self, "fill_value"):
            params["fill_value"] = self.fill_value
        return params

    def get_sentences(self, text, lang='en'):
        return sent_tokenize(text, self.LANGS.get(lang, 'english'))

class SwapWordsTransform(NLPTransform):
    """ Swap words next to each other """
    def __init__(self, swap_distance=1, swap_probability=0.1, always_apply=False, p=0.5):
        """  
        swap_distance - distance for swapping words
        swap_probability - probability of swapping for one word
        """
        super(SwapWordsTransform, self).__init__(always_apply, p)
        self.swap_distance = swap_distance
        self.swap_probability = swap_probability
        self.swap_range_list = list(range(1, swap_distance+1))

    def apply(self, data, **params):
        text, lang = data
        words = text.split()
        words_count = len(words)
        if words_count <= 1:
            return text, lang

        new_words = {}
        for i in range(words_count):
            if random.random() > self.swap_probability:
                new_words[i] = words[i]
                continue
    
            if i < self.swap_distance:
                new_words[i] = words[i]
                continue
    
            swap_idx = i - random.choice(self.swap_range_list)
            new_words[i] = new_words[swap_idx]
            new_words[swap_idx] = words[i]

        return ' '.join([v for k, v in sorted(new_words.items(), key=lambda x: x[0])]), lang
    
class CutOutWordsTransform(NLPTransform):
    """ Remove random words """
    def __init__(self, cutout_probability=0.05, always_apply=False, p=0.5):
        super(CutOutWordsTransform, self).__init__(always_apply, p)
        self.cutout_probability = cutout_probability

    def apply(self, data, **params):
        text, lang = data
        words = text.split()
        words_count = len(words)
        if words_count <= 1:
            return text, lang

        new_words = []
        for i in range(words_count):
            if random.random() < self.cutout_probability:
                continue
            new_words.append(words[i])

        if len(new_words) == 0:
            return words[random.randint(0, words_count-1)], lang

        return ' '.join(new_words), lang

In [11]:
f = open("/kaggle/input/zindiswahili3/stopswahili.txt", "r")
sw_stop = []
for x in f:
  sw_stop.append(re.sub('\n','',x))

sw_stop2 = "/kaggle/input/zindiswahili3/Common Swahili Stop-words.csv"
sw_stop2 = pd.read_csv(sw_stop2)
sw_stop2 = ((sw_stop2.to_numpy()).reshape((1,len(sw_stop2))))
sw_stop2= sw_stop2.tolist()
sw_stop = sw_stop+sw_stop2[0]

def remove_stop0(tweet):
    query = tweet
    stopwords1 = sw_stop
    query = query.split()

    resultwords  = [word for word in query if word.lower() not in stopwords1]
    #result = ' '.join(resultwords)
    return ' '.join(resultwords)

def remove_stop(tweet):
    query = tweet
    stopwords1 = ['na', 'ya', 'wa', 'za', 'kwa', 'la', 'vya', 'ni', 'kila', 'so', 'kwenye', 'si'] #, 'asante','nzuri','wazuri']
    query = query.split()

    resultwords  = [word for word in query if word.lower() not in stopwords1]
    #result = ' '.join(resultwords)
    return ' '.join(resultwords)



def processText(tweet):
    
  tweet = tweet.lower()                #convert text to lower-case
  tweet = re.sub('â€˜','',tweet)    # remove the text â€˜ which appears to occur flequently
  tweet = re.sub('â€™','',tweet)    # remove the text â€™ which appears to occur flequently
  tweet = re.sub('â€œ', '', tweet)
  tweet = re.sub('Â', '', tweet)
  tweet = re.sub('â€','',tweet)
  tweet = re.sub('â€“','',tweet)
    

  tweet = word_tokenize(tweet)      # remove repeated characters (helloooooooo into hello)
  return ' '.join(tweet)

In [12]:
df['comment_text']  = df['Tweets']
df['comment_text'] = df['comment_text'].apply(processText) 
df['comment_text'] = df['comment_text'].apply(remove_stop0)   
df['comment_text'] = df['comment_text'].apply(remove_stop)  


df_test['Tweets'] = df_test['Tweets'].apply(processText)
df_test['Tweets'] = df_test['Tweets'].apply(remove_stop0)  
df_test['Tweets'] = df_test['Tweets'].apply(remove_stop)  

In [13]:
# steeming the words to root however did help much

def stemmi_sw(data):
    
    data = data.replace(' una', " ")
    data = data.replace(' nina', " ")
    data = data.replace(' tuna'," ")
    data = data.replace(' nime'," ")
    data = data.replace(' niki'," ")
    data = data.replace(' ume'," ")
    data = data.replace(' nawa'," ")
    data = data.replace(' yata'," ")
    data = data.replace(' kum'," ")
    data = data.replace(' wana'," ")
    data = data.replace(' ana'," ")
    data = data.replace(' ame'," ")
    data = data.replace(' kui'," ")
    data = data.replace(' waku'," ")
    data = data.replace(' tuna'," ")
    data = data.replace(' tuta'," ")
    data = data.replace(' tu'," ")
    data = data.replace(' zime'," ")
    data = data.replace(' nili'," ")
    data = data.replace(' usi'," ")
    data = data.replace(' ime'," ")
    data = data.replace(' li'," ")
    data = data.replace(' siku '," juma ")
    data = data.replace(' siku'," ")

    
    
    return data 



In [14]:


# Function to clean the review text and remove all the unnecessary elements.

def clean_review_text(text):
    text = text.lower()  # covert the text to lowercase
    text = re.sub('<.*?>','',text).strip() # remove html chars
    text = re.sub('\[|\(.*\]|\)','', text).strip() # remove text in square brackets and parenthesis
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove punctuation marks
    text = re.sub("(\\W)"," ",text).strip() # remove non-ascii chars
    text = re.sub('\S*\d\S*\s*','', text).strip()  # remove words containing numbers
    return text.strip()



In [15]:
df.comment_text = df.comment_text.astype(str)
df.comment_text = df.comment_text.apply(clean_review_text)
df_test.Tweets = df_test.Tweets.astype(str)
df_test.Tweets = df_test.Tweets.apply(clean_review_text)
df.comment_text.head()

0    chuga tunakutana kesho nyamachoma festival nan...
1    asante sirari jimbo tarime vijijini upendo usi...
2        leo nimepata kitambulisho changu taifa asante
3                   mgema akisifiwa tembo hulitia maji
4    ee mwenyezi mungu msamehe umrehemu umuafu msam...
Name: comment_text, dtype: object

In [16]:
X = df.comment_text
y = df.drop(['ID','Tweets', 'comment_text'],axis = 1)

In [17]:
# Split the dataset into test and train
from sklearn.model_selection import train_test_split
seed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer


word_vectorizer = TfidfVectorizer(
    strip_accents='unicode',     
    analyzer='word',            
    token_pattern=r'\w{1,}',    
    ngram_range=(1, 3),         
    stop_words='english',
    sublinear_tf=True)

word_vectorizer.fit(X_train)    # Fiting it on Train
train_word_features = word_vectorizer.transform(X_train)

In [19]:


## transforming the train and test datasets
X_train_transformed = word_vectorizer.transform(X_train)
X_test_transformed = word_vectorizer.transform(X_test)


# # Print the shape of each dataset.
print('X_train_transformed', X_train_transformed.shape)
print('y_train', y_train.shape)
print('X_test_transformed', X_test_transformed.shape)
print('y_test', y_test.shape)



X_train_transformed (1810, 41022)
y_train (1810, 2)
X_test_transformed (453, 41022)
y_test (453, 2)


# Machine learning Model Logistic Classification

Prepape test set

In [20]:
df_test['comment_text'] = df_test['Tweets']
df_test.comment_text = df_test.comment_text.astype(str)
df_test.comment_text = df_test.comment_text.apply(clean_review_text)
X_test_pred = df_test.comment_text

In [21]:
n_folds = 5
kfold = model_selection.StratifiedKFold(n_splits= n_folds, shuffle=True,random_state=seed)
valid_pred = pd.DataFrame()
splits = list(kfold.split(df, df.Labels))
probabilitiesF = np.zeros((len(df_test),3))



for i,(train_index,test_index) in enumerate(splits):

    trdf = df.iloc[train_index,:]
    trdf2 = df.iloc[train_index,:]
    trdf3 = df.iloc[train_index,:]
    classes = [-1, 0, 1]
    weights = list(compute_class_weight(class_weight='balanced',y=trdf['Labels'],classes = classes))
    cweights = dict(zip(classes, weights))

    transform = SwapWordsTransform(p=1.0, swap_distance=3, swap_probability=0.6)
    tr=CutOutWordsTransform(p=1.0, cutout_probability=0.001)

    lang = 'en'
    for j in list(trdf.index):
        text = trdf['comment_text'][j]
        #print('text before', trdf['text'][j])
        text=transform(data=(text, lang))['data'][0]
        trdf['comment_text'][j]=tr(data=(text, lang))['data'][0]

    transform2 = SwapWordsTransform(p=1.0, swap_distance=1, swap_probability=0.7)
    tr2=CutOutWordsTransform(p=1.0, cutout_probability=0.01)

    lang = 'en'
    for j in list(trdf2.index):

        text = trdf2['comment_text'][j]
        #print('text before', trdf['text'][j])
        text=transform2(data=(text, lang))['data'][0]
        trdf2['comment_text'][j]=tr2(data=(text, lang))['data'][0]




    transform3 = SwapWordsTransform(p=1.0, swap_distance=3, swap_probability=0.6)
    tr3=CutOutWordsTransform(p=1.0, cutout_probability=0.01)

    lang = 'en'
    for j in list(trdf3.index):
        text = trdf3['comment_text'][j]
        text=transform3(data=(text, lang))['data'][0]
        trdf3['comment_text'][j]=tr3(data=(text, lang))['data'][0]


    neg=trdf[trdf['Labels']==-1]
    neg2=trdf2[trdf2['Labels']==-1]
    one=trdf3[trdf3['Labels']==1]
    one=one.sample(frac=0.5).reset_index(drop=True)



    train_real=pd.concat([neg, df.iloc[train_index,:], one, neg2],axis=0).sample(frac=1).reset_index(drop=True) #0.32 ens

    X_train = train_real.comment_text
    y_train  = train_real.Labels

    #print(y_train)

    X_test = df.iloc[test_index,:].comment_text
    y_test  = df.iloc[test_index,:].Labels

    word_vectorizer = TfidfVectorizer(
                strip_accents='unicode',     
                analyzer='word',            
                token_pattern=r'\w{1,}',    
                ngram_range=(1, 3),         
                stop_words='english',
                sublinear_tf=True)

    word_vectorizer.fit(X_train)    # Fiting it on Train
    train_word_features = word_vectorizer.transform(X_train)

        ## transforming the train and test datasets
    X_train_transformed = word_vectorizer.transform(X_train)
    X_test_transformed = word_vectorizer.transform(X_test)


    label  = trdf.Labels #dtrain[:,32]

    time1 = time.time()
    # logistic regression
    log_reg = LogisticRegression(C = 10, penalty='l2', solver = 'liblinear', random_state=seed)


    classifier_ovr_log = OneVsRestClassifier(log_reg)
    classifier_ovr_log.fit(X_train_transformed, y_train)


    time_taken = time.time() - time1
    print('Time Taken: {:.2f} seconds'.format(time_taken))

    y_train_pred_proba = classifier_ovr_log.predict_proba(X_train_transformed)
    y_test_pred_proba = classifier_ovr_log.predict_proba(X_test_transformed)


    roc_auc_score_train = roc_auc_score(y_train, y_train_pred_proba,multi_class="ovr", average='weighted')
    roc_auc_score_test = roc_auc_score(y_test, y_test_pred_proba,multi_class="ovr", average='weighted')

    print("ROC AUC Score Train:", roc_auc_score_train)
    print("ROC AUC Score Test:", roc_auc_score_test)



    classifier = classifier_ovr_log

    X_test_transformed_pred = word_vectorizer.transform(X_test_pred)

    y_test_pred = classifier.predict_proba(X_test_transformed_pred) 

    probabilitiesF = probabilitiesF + y_test_pred


    y_test_pred_df = pd.DataFrame(y_test_pred,columns=['-1', '0', '1'])
    df_test['Labels'] = y_test_pred_df.idxmax(axis = 1)
    df_test.to_csv('submissiontestkfold11'+str(i)+'.csv', index = False)


    print('********')
    print('Model', i)
    print('***********')



Time Taken: 0.11 seconds
ROC AUC Score Train: 0.999981093304947
ROC AUC Score Test: 0.6836508698786434
********
Model 0
***********
Time Taken: 0.12 seconds
ROC AUC Score Train: 0.9999841684811859
ROC AUC Score Test: 0.635143309682947
********
Model 1
***********
Time Taken: 0.11 seconds
ROC AUC Score Train: 0.9999717995269936
ROC AUC Score Test: 0.6132354849517041
********
Model 2
***********
Time Taken: 0.18 seconds
ROC AUC Score Train: 0.9999830816447992
ROC AUC Score Test: 0.6266698144992682
********
Model 3
***********
Time Taken: 0.12 seconds
ROC AUC Score Train: 0.9999872830539466
ROC AUC Score Test: 0.6134151562892031
********
Model 4
***********


In [22]:
probabilitiesF= probabilitiesF/(n_folds)
y_test_pred_df = pd.DataFrame(probabilitiesF,columns=['-1', '0', '1'])
df_test['Labels'] = y_test_pred_df.idxmax(axis = 1)
df_test.to_csv('submissiontestkfoldTotal.csv', index = False)
#df_test.to_csv('submissiontestkfold1.csv', index = False)

probabilitiesLogistic = 0.1 * probabilitiesF    # to ensemble with the other model

In [23]:
probabilitiesLogistic

array([[0.02433156, 0.04483966, 0.03082877],
       [0.01816151, 0.01620924, 0.06562925],
       [0.00474935, 0.01855272, 0.07669793],
       ...,
       [0.00984355, 0.06991012, 0.02024633],
       [0.00965848, 0.06351324, 0.02682829],
       [0.01117185, 0.07272916, 0.01609898]])

# CAT  BOOST TO ENSEMBLE

In [24]:
#function to generate categorical

def cat_fun(x):
    return x+1

df_cat['labels'] = df_cat['Labels'].map(cat_fun) # this is necessary for transformer
target = 'labels'


In [25]:
df_cat['stemtweet'] = df_cat['Tweets'].apply(stemmi_sw) 
df_test_cat['stemtweet'] = df_test_cat['Tweets'].apply(stemmi_sw)


df_cat['num_char'] = df_cat.Tweets.apply(len)
df_cat['num_word'] = df_cat.Tweets.apply(lambda x: len(x.split()))
df_cat['word_ratio'] = df_cat.num_char / (df_cat.num_word +1)
df_cat['num_punctuation'] = df_cat.Tweets.apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
df_cat['num_title_word'] = df_cat.Tweets.apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
df_cat['num_upper_case_word'] = df_cat.Tweets.apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
df_cat['num_char_stemmered'] = df_cat.stemtweet.apply(len)
df_cat['stem_word_ratio'] = df_cat.num_char_stemmered / (df_cat.num_word +1)


df_test_cat['num_char'] = df_test_cat.Tweets.apply(len)
df_test_cat['num_word'] = df_test_cat.Tweets.apply(lambda x: len(x.split()))
df_test_cat['word_ratio'] = df_test_cat.num_char / (df_test_cat.num_word +1)
df_test_cat['num_punctuation'] = df_test_cat.Tweets.apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
df_test_cat['num_title_word'] = df_test_cat.Tweets.apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
df_test_cat['num_upper_case_word'] = df_test_cat.Tweets.apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
df_test_cat['num_char_stemmered'] = df_test_cat.stemtweet.apply(len)
df_test_cat['stem_word_ratio'] = df_test_cat.num_char_stemmered / (df_test_cat.num_word +1)

nlp_features = ['num_char','num_word','word_ratio','num_punctuation','num_title_word','num_upper_case_word', 'num_char_stemmered', 'stem_word_ratio' ]

In [26]:
num_val = ['num_word','word_ratio','num_punctuation','num_title_word','num_upper_case_word']  # try and error selection

use_features = num_val + ['English_tweet']

In [27]:
n_folds = 5
stkfold = model_selection.StratifiedKFold(n_splits= n_folds, shuffle=True,random_state=seed)

splits = list(stkfold.split(df_cat, df_cat.labels))

parameters = {
        'learning_rate':0.2,
        'iterations':500,
        'eval_metric':'Accuracy'
        }


# model = cb.CatBoostClassifier(task_type= 'GPU',loss_function='MultiClass',random_seed=seed,
#                               early_stopping_rounds=100,**parameters)

model = cb.CatBoostClassifier(task_type= 'CPU',loss_function='MultiClass',
                              text_features= ['English_tweet'],random_seed=seed,
                              early_stopping_rounds=100,**parameters)

for i,(train_index,test_index) in enumerate(splits):
    x_train,x_test = df_cat.loc[train_index,use_features],df_cat.loc[test_index,use_features]
    y_train,y_test = df_cat.loc[train_index,'Labels'],df_cat.loc[test_index,'Labels']
    
#     model.fit(x_train, y_train,use_best_model=True,
#               eval_set=(x_test,y_test),verbose= 50)
    
    _train = Pool(x_train, label=y_train,cat_features=None,text_features=['English_tweet'])
    _valid = Pool(x_test, label=y_test, cat_features=None,text_features=['English_tweet'])
    
    # Train
    model.fit(_train, eval_set=_valid,use_best_model=True,verbose= 50)
        
    x_test_preds_prob = model.predict_proba(x_test)
    x_test_preds = model.predict(x_test)
    

    score = metrics.accuracy_score(y_true=y_test, y_pred=x_test_preds)
    print(f"Fold {i} Accuracy: {score} \n")
    
    mark_classes = [f't1 {x}' for x in model.classes_]
    classes = model.classes_
    
    # Out-of-Fold Predictions
    df_cat.loc[test_index,mark_classes] = x_test_preds_prob
    

    probcols = [f'{each} {i}' for each in classes]
    
    predictions = model.predict_proba(df_test_cat[use_features])
    df_test_cat.loc[:,probcols] = predictions

0:	learn: 0.5944751	test: 0.5894040	best: 0.5894040 (0)	total: 527ms	remaining: 4m 22s
50:	learn: 0.6337017	test: 0.6136865	best: 0.6203091 (47)	total: 25.7s	remaining: 3m 46s
100:	learn: 0.6861878	test: 0.6092715	best: 0.6203091 (47)	total: 51.2s	remaining: 3m 22s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6203090508
bestIteration = 47

Shrink model to first 48 iterations.
Fold 0 Accuracy: 0.6203090507726269 

0:	learn: 0.5928177	test: 0.5916115	best: 0.5916115 (0)	total: 464ms	remaining: 3m 51s
50:	learn: 0.6381215	test: 0.5960265	best: 0.6004415 (47)	total: 25.5s	remaining: 3m 44s
100:	learn: 0.6751381	test: 0.5960265	best: 0.6048565 (54)	total: 51s	remaining: 3m 21s
150:	learn: 0.7066298	test: 0.5982340	best: 0.6048565 (54)	total: 1m 16s	remaining: 2m 55s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6048565121
bestIteration = 54

Shrink model to first 55 iterations.
Fold 1 Accuracy: 0.6048565121412803 

0:	learn: 0.5939227	test: 0.5

In [28]:
df_cat['prediction'] = df_cat[mark_classes].idxmax(axis='columns').str.replace('t1 ','').astype(int)
final_score = metrics.accuracy_score(y_true=df_cat.Labels,y_pred=df_cat['prediction'])
print(f'Accuracy score: {final_score}')

# store to the dataframes

for each in classes:
    foldcols = [f'{each} {x}' for x in range(n_folds)]
    df_test_cat[f't1 {each}'] = df_test_cat[foldcols].mean(axis='columns') # mean probability

df_test_cat['Labels'] = df_test_cat[mark_classes].idxmax(axis='columns').str.replace('t1 ','').astype(int)

submission = pd.DataFrame({'ID':df_test_cat.ID,'Labels':df_test_cat.Labels})
submission.to_csv('submission11OOP.csv',index=False)


Accuracy score: 0.6146707909854175


In [29]:
# Purposely to ensemble
ens_models = []      
ens_models += mark_classes
ens_models

['t1 -1', 't1 0', 't1 1']

Motivation https://towardsdatascience.com/going-global-how-to-multi-task-in-multiple-languages-with-the-mt5-transformer-892617cd890c

In [30]:
# this is special for transformer 
df_cat['text'] = df_cat.English_tweet.values
df_test_cat['text'] = df_test_cat.English_tweet.values

In [31]:
# function to set seed 
def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
seed_all(seed)


from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [32]:
## import a model

from simpletransformers.classification import ClassificationModel,ClassificationArgs

model_type = "bert"   # i tested other model poor performance
model_name =  "bert-base-cased"


cuda_available = torch.cuda.is_available() # check CUDA availablity

# Most of this parameter are default from Simpletransfomer Notebook
model_args = {
    'num_train_epochs': 5,
    'learning_rate': 2e-5,
    'manual_seed':seed,
    'fp16': True, 	
    'max_seq_length': 70, 
    'train_batch_size': 128,  
    'output_dir': "outputs/",  



    'do_lower_case': False,  
    'early_stopping_consider_epochs': False, 	
    'early_stopping_delta': 0, 	
    'early_stopping_metric': 'eval_loss', 	
    'early_stopping_metric_minimize': True, 	
    'early_stopping_patience':3,	
    'use_early_stopping': False,

    'reprocess_input_data': True, 

    'save_model_every_epoch': True , 	
    'save_optimizer_and_scheduler': False, 	
    'save_steps':-1, 
    
    'scheduler': 'linear_schedule_with_warmup', 

    
    'overwrite_output_dir':True, 

    'sliding_window': False,
    'stride': 0.8,

    'regression': False, 
    
}

specific_args = {
    'num_labels': df_cat[target].nunique(), 

}
model_args,specific_args


n_folds = 5
stkfold = model_selection.StratifiedKFold(n_splits= n_folds, shuffle=True,random_state=seed)

splits = list(stkfold.split(df_cat, df_cat.labels))

for i,(train_index,test_index) in enumerate(splits):
    
    cols = ['labels','text']
    x_train = df_cat.loc[train_index,cols].copy().reset_index(drop=True)
    x_test = df_cat.loc[test_index,cols].copy().reset_index(drop=True)
    
    
    model = ClassificationModel(model_type, 
                                model_name, 
                                use_cuda=cuda_available,
                                args=model_args,**specific_args)

    
    
    classes = list(range(specific_args['num_labels']))
    
    
    # Train model
    model.train_model(train_df = x_train)

    result, model_outputs, wrong_predictions = model.eval_model(x_test)
    
    print(f"fold {i} Result:",result)
    

    x_test_preds,x_test_preds_rawout =  model.predict(x_test['text'].values.tolist())
    
    score = metrics.accuracy_score(y_true=x_test.labels, y_pred=x_test_preds)
    print(f"Fold {i} Accuracy: {score} \n")    
    
    mark_classes = [f't2 {x}' for x in classes]
    
    # Out-of-Fold Predictions
    df_cat.loc[test_index,mark_classes] = x_test_preds_rawout
    
    
    # predictions on testset
    predictions, raw_outputs = model.predict(df_test_cat['text'].values.tolist())
    
    # probabilities results
    probcols = [f'{each} pr {i}' for each in classes]
    df_test_cat.loc[:,probcols] = raw_outputs
    
    print('***************')
    print('End Train :', i)
    


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

...

Fold 4 Accuracy: 0.5973451327433629 



  0%|          | 0/755 [00:00<?, ?it/s]

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction


  0%|          | 0/95 [00:00<?, ?it/s]

***************
End Train : 4


In [33]:
df_cat['prediction'] = df_cat[mark_classes].idxmax(axis='columns').str.replace('t2 ','').astype(int)
final_score = metrics.accuracy_score(y_true=df_cat.labels,y_pred=df_cat['prediction'])
print(f'Accuracy score: {final_score}')
    
def min_cat(x):
    return x-1

for each in classes:
    foldcols = [f'{each} pr {x}' for x in range(n_folds)]
    df_test_cat[f't2 {each}'] = df_test_cat[foldcols].mean(axis='columns') 
    
df_test_cat['labels'] = df_test_cat[mark_classes].idxmax(axis='columns').str.replace('t2 ','').astype(int)
df_test_cat['Labels'] = df_test_cat['labels'] = df_test_cat['labels'].map(min_cat).to_numpy(copy=True)
df_test_cat[['ID','Labels']].to_csv(f'deepsubmission.csv',index=False)

Accuracy score: 0.6155545735749006


In [34]:
ens_models += mark_classes
use_features = ens_models + ['English_tweet'] # this append all previous out of prediction to the feature

In [35]:
n_folds = 5
stkfold = model_selection.StratifiedKFold(n_splits= n_folds, shuffle=True,random_state=seed)

splits = list(stkfold.split(df_cat, df_cat.labels))

parameters = {
#         'iterations': 100, 
        'learning_rate': 0.2,
        'iterations':500,
#         'l2_leaf_reg':50,
#         'loss_function':'Logloss',
        'eval_metric':'Accuracy'
        }


# model = cb.CatBoostClassifier(task_type= 'GPU',loss_function='MultiClass',random_seed=seed,
#                               early_stopping_rounds=100,**parameters)

model = cb.CatBoostClassifier(task_type= 'CPU',loss_function='MultiClass',
                              text_features= ['English_tweet'],random_seed=seed,
                              early_stopping_rounds=100,**parameters)

for i,(train_index,test_index) in enumerate(splits):
    x_train,x_test = df_cat.loc[train_index,use_features],df_cat.loc[test_index,use_features]
    y_train,y_test = df_cat.loc[train_index,'Labels'],df_cat.loc[test_index,'Labels']
    
    
    _train = Pool(x_train, label=y_train,cat_features=None,text_features=['English_tweet'])
    _valid = Pool(x_test, label=y_test, cat_features=None,text_features=['English_tweet'])
    

    model.fit(_train, eval_set=_valid,use_best_model=True,verbose= 50)
        
    x_test_preds_prob = model.predict_proba(x_test)
    x_test_preds = model.predict(x_test)
        
    

    score = metrics.accuracy_score(y_true=y_test, y_pred=x_test_preds)
    print(f"Fold {i} Accuracy: {score} \n")
    
    classes = model.classes_
    # Out-of-Fold Predictions
    df_cat.loc[test_index,classes] = x_test_preds_prob
    
    probcols = [f'{each} {i}' for each in classes]
    
    predictions = model.predict_proba(df_test_cat[use_features])
    df_test_cat.loc[:,probcols] = predictions

0:	learn: 0.6121547	test: 0.6225166	best: 0.6225166 (0)	total: 549ms	remaining: 4m 33s
50:	learn: 0.6662983	test: 0.6490066	best: 0.6534216 (30)	total: 27s	remaining: 3m 57s
100:	learn: 0.7176796	test: 0.6467991	best: 0.6534216 (30)	total: 52.8s	remaining: 3m 28s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6534216336
bestIteration = 30

Shrink model to first 31 iterations.
Fold 0 Accuracy: 0.6534216335540839 

0:	learn: 0.6121547	test: 0.5739514	best: 0.5739514 (0)	total: 470ms	remaining: 3m 54s
50:	learn: 0.6773481	test: 0.6158940	best: 0.6203091 (9)	total: 25.4s	remaining: 3m 43s
100:	learn: 0.7292818	test: 0.6026490	best: 0.6203091 (9)	total: 50.6s	remaining: 3m 19s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6203090508
bestIteration = 9

Shrink model to first 10 iterations.
Fold 1 Accuracy: 0.6203090507726269 

0:	learn: 0.5983425	test: 0.6026490	best: 0.6026490 (0)	total: 468ms	remaining: 3m 53s
50:	learn: 0.6839779	test: 0.6357616

In [36]:
df_cat['prediction'] = df_cat[classes].idxmax(axis='columns')
final_score = metrics.accuracy_score(y_true=df_cat.Labels,y_pred=df_cat['prediction'])
print(f'Accuracy score: {final_score}')
for each in classes:
    foldcols = [f'{each} {x}' for x in range(n_folds)]
    df_test_cat[each] = df_test_cat[foldcols].mean(axis='columns') # mean probability

df_test_cat['Labels'] = df_test_cat[classes].idxmax(axis='columns')

submission = pd.DataFrame({'ID':df_test_cat.ID,'Labels':df_test_cat.Labels})
submission.to_csv('final_zindi_3rdplace.csv',index=False)
submission.head()

Accuracy score: 0.6341140079540433


,ID,Labels
0,4,1
1,7,1
2,9,1
3,10,1
4,15,0


# MT5

I couldnt be able to finish this.

In [37]:
#df_test_cat.head()